# Semi-inducibility Problem on 4 Vertices

This notebook contains calculations for the semi-inducibility problem on all 4 vertex red-blue colored graphs. To run these calculations, use the modified version of sage from
https://github.com/bodnalev/sage

The following cell sets up a theory to calculate with blue-red colored graphs and transform the semi-inducibility questions to graph theory. The rest of the notebook is organized as follows: 

 - Section 1 contains the main optimization, including the constructions and the corresponding upper bounds.

 - Section 2 contains methods to test stability of the obtained results.

 - Section 3 contains a single cell verifying all the certificates. It can be run independently from the rest of the notebook.

 - Section 4 contains code that exports pickled certificates to human readable text files. Additionally there is an interactive code to inspect the certificate data.

In [1]:
G = GraphTheory
G.printlevel(0)
BG = Theory("BlueGraph", "blue_edges")
CG = combine("RedBlueGraph", G, BG, symmetries=FullSymmetry)
CG.exclude(CG(2, edges=[[0, 1]], blue_edges=[[0, 1]]))

def check_blocks(flg, do, dont):
    eds = flg.blocks("edges")
    for ee in do:
        if ee not in eds:
            return False
    for ee in dont:
        if ee in eds:
            return False
    return True

def get_targ(ff):
    vn = ff.size()
    eddo = ff.blocks("edges")
    eddont = ff.blocks("blue_edges")
    pats = G.p(vn, edges=eddo, edges_m=eddont)
    targ = 0
    for xx in pats.compatible_flags():
        coef = 0
        for eperm in itertools.permutations(range(vn)):
            xxpermed = xx.subflag(eperm)
            if check_blocks(xxpermed, eddo, eddont):
                coef += 1
        targ += coef*xx
    return targ*QQ(1/(factorial(vn)))

def gen_nont(n):
    ret = []
    for ff in CG.generate(n):
        eds = ff.blocks("edges")
        oeds = ff.blocks("blue_edges")
        if len(eds)!=0 and len(oeds)!=0:
            if len(eds) + len(oeds) != binomial(n, 2):
                if len(set(itertools.chain(*(list(eds) + list(oeds)))))==n:
                    ret.append(ff)
    return ret

def flip_colors(fl):
    return CG(fl.size(), edges=fl.blocks("blue_edges"), blue_edges=fl.blocks("edges"))

# For easier presentation, these flips guarantee no loops in blowup patterns
gen4 = [(xx if ii not in [4, 8, 9, 10, 14, 17] else flip_colors(xx)) for ii,xx in enumerate(gen_nont(4))]
targ4 = [get_targ(xx) for xx in gen4]

print("The unique semi-inducibility graphs on 4 vertices are")
for xx in gen4:
    print(f"\n{xx}")

The unique semi-inducibility graphs on 4 vertices are

Flag on 4 points, ftype from () with edges=(02), blue_edges=(13)

Flag on 4 points, ftype from () with edges=(01), blue_edges=(02 03)

Flag on 4 points, ftype from () with edges=(01), blue_edges=(02 13)

Flag on 4 points, ftype from () with edges=(02), blue_edges=(01 13)

Flag on 4 points, ftype from () with edges=(02 03 13), blue_edges=(01)

Flag on 4 points, ftype from () with edges=(01 02 03), blue_edges=(13)

Flag on 4 points, ftype from () with edges=(02), blue_edges=(01 03 13)

Flag on 4 points, ftype from () with edges=(03), blue_edges=(02 12 13)

Flag on 4 points, ftype from () with edges=(02 03 12 13), blue_edges=(01)

Flag on 4 points, ftype from () with edges=(01 02 12 13), blue_edges=(03)

Flag on 4 points, ftype from () with edges=(01 13), blue_edges=(02 03)

Flag on 4 points, ftype from () with edges=(01 03), blue_edges=(02 13)

Flag on 4 points, ftype from () with edges=(02 03), blue_edges=(12 13)

Flag on 4 points, 

## Section 1 - Constructions and upper bounds

This section contains a list of constructions used to help the rounding. The standard rounding method works for all cases except $H_3$ and $H_5$. 

Since the semi-inducibility problem for $H_5$ is symmetrizable, Subsection 1.1 calculates in the multi-partite theory, establishing a matching construction and upper bound. 

Subsection 1.2 discusses the remaining case $H_3$, by providing a strong construction and a numerically close upper bound.

In [2]:
def kn_edges(n):
    return list(itertools.combinations(range(n), 2))

constructions = [
    G.blowup_construction(5, 1, edges={(0, 0): 1/2}), 
    G.blowup_construction(4, 1, edges={(0, 0): 1/3}), 
    G.blowup_construction(4, 1, edges={(0, 0): 1/3}), 
    None, 
    G.blowup_construction(4, 3, edges=kn_edges(3)), 
    None, 
    G.blowup_construction(6, 2, edges=[[0, 1]]), 
    G.blowup_construction(4, 1, edges={(0, 0): 1/4}), 
    G.blowup_construction(4, 3, edges=kn_edges(3)), 
    G.blowup_construction(7, 5, edges=kn_edges(5)), 
    G.blowup_construction(5, 2, edges=[[0, 1]]),    
    G.blowup_construction(4, 2, edges=[[0, 1]]), 
    G.blowup_construction(5, 2, edges=[[0, 1]]), 
    G.blowup_construction(5, 2, edges=[[0, 1]]),
    G.blowup_construction(5, 2, edges=[[0, 1]]),
    G.blowup_construction(4, 2, edges={(0, 1): 2/3}),
    G.blowup_construction(5, 2, edges=[[0, 1]]),
    G.blowup_construction(5, 2, edges=[[0, 1]])
]

In [3]:
for index in range(len(gen4)):
    constr = constructions[index]
    targ = get_targ(gen4[index])
    if constr!=None:
        size = constr.size()
        dens = constr.density(targ)
        bound = G.optimize(targ, size, denom=2**20, exact=True, construction=constr, file="certificates/semiind"+str(index))
        print("\nIndex {} is {} the constr gives {}, upper bound is {}".format(index, gen4[index], dens, bound))
    else:
        bound = G.optimize(targ, 7, exact=False, file="certificates/fp_semiind"+str(index))
        print("\nIndex {} is {} the upper bound is {}".format(index, gen4[index], bound))


Index 0 is Flag on 4 points, ftype from () with edges=(02), blue_edges=(13) the constr gives 1/4, upper bound is 1/4

Index 1 is Flag on 4 points, ftype from () with edges=(01), blue_edges=(02 03) the constr gives 4/27, upper bound is 4/27

Index 2 is Flag on 4 points, ftype from () with edges=(01), blue_edges=(02 13) the constr gives 4/27, upper bound is 4/27

Index 3 is Flag on 4 points, ftype from () with edges=(02), blue_edges=(01 13) the upper bound is 0.15008340915196144

Index 4 is Flag on 4 points, ftype from () with edges=(02 03 13), blue_edges=(01) the constr gives 4/27, upper bound is 4/27

Index 5 is Flag on 4 points, ftype from () with edges=(01 02 03), blue_edges=(13) the upper bound is 0.1500644984603527

Index 6 is Flag on 4 points, ftype from () with edges=(02), blue_edges=(01 03 13) the constr gives 1/8, upper bound is 1/8

Index 7 is Flag on 4 points, ftype from () with edges=(03), blue_edges=(02 12 13) the constr gives 27/256, upper bound is 27/256

Index 8 is Flag

### Subsection 1.1 - Multipartite construction and upper bound for $H_5$

This calculation is done over graphs with `G(3, edges=[[0, 1]])` excluded, due to the problem being symmetrizable.

In [2]:
MPG = Theory("MultiPartiteGraph")
MPG.exclude(MPG(3, edges=[[0, 1]]))
MPG.printlevel(0)
var("x")
RF = RealField(prec=100)
alpha_real = RF(solve(x^2 - 13/7*x + 4/7==0, x)[0].rhs())
R.<alpha> = NumberField(x^2 - 13/7*x + 4/7, embedding=alpha_real)
cons = MPG.blowup_construction(8, [alpha/4, alpha/4, alpha/4, alpha/4, 1-alpha], edges=list(itertools.combinations(range(5), 2)))
mp_targ = (1/4)*MPG(4, edges=[[0, 1], [0, 2], [0, 3]]) + (1/6)*MPG(4, edges=[[0, 1], [0, 2], [0, 3], [1, 2], [1, 3]])
dens = cons.density(mp_targ)
print("For index 5, construction provides density {}".format(dens))
print("Here, alpha is the root of {} around {}".format(alpha.minpoly(), alpha.n()))

For index 5, construction provides density 171/3136*alpha + 101/784
Here, alpha is the root of x^2 - 13/7*x + 4/7 around 0.389297540337804


In [18]:
bound = MPG.optimize(mp_targ, 8, construction=cons, exact=True, denom=2**20, kernel_denom=2**20, file="certificates/semiind5")
print("For index 5, the multi-partite upper bound is {}".format(bound))
print("Here, alpha is the same root of {} around {}".format(alpha.minpoly(), alpha.n()))

For index 5, the multi-partite upper bound is 171/3136*alpha + 101/784
Here, alpha is the same root of x^2 - 13/7*x + 4/7 around 0.389297540337804


In [3]:
val = 171/3136 * solve(x^2-13/7*x+4/7==0, x)[0].rhs() + 101/784
print("The final bound and construction equals to {} around {}".format(val, val.n()))

The final bound and construction equals to -171/43904*sqrt(57) + 7879/43904 around 0.150054170726328


### Subsection 1.2 - Construction and numeric upper bound for $H_3$

First, the polynomial $p(\beta, \gamma)$ is calculated, with the relations $\beta, \gamma$ must satisfy at any local optimum.
Then the solutions of these relations are compared.

In [4]:
print("The optimal parameters in ??? must satisfy the following")
R.<gamma, beta> = PolynomialRing(QQ, 2, order="lex")
cons = G.blowup_construction(4, [beta, 1-beta], edges={(0, 0): gamma, (1, 1): 1})
expr = cons.density(targ4[3])
ders = [expr.derivative(beta), expr.derivative(gamma)]
I = R.ideal(ders)
Gb = I.groebner_basis()
print(Gb[-1], "= 0")

R.<beta, gamma> = PolynomialRing(QQ, 2, order="lex")
cons = G.blowup_construction(4, [beta, 1-beta], edges={(0, 0): gamma, (1, 1): 1})
expr = cons.density(targ4[3])
ders = [expr.derivative(beta), expr.derivative(gamma)]
I = R.ideal(ders)
Gb = I.groebner_basis()
print(Gb[-1], "= 0")

The optimal parameters in ??? must satisfy the following
beta^5 - 10/3*beta^4 + 2521/576*beta^3 - 407/144*beta^2 + 43/48*beta - 1/9 = 0
gamma^4 - 37/16*gamma^3 + 57/32*gamma^2 - 9/16*gamma + 1/16 = 0


In [5]:
var("x y")
print("The roots provide:")
for ssg in solve(Gb[-1].subs(gamma=x)==0, x):
    ngamma = real_part(ssg.rhs().n())
    for ss in solve(Gb[0].subs(beta=y)==0, y):
        nbeta = ss.rhs().subs(gamma=ngamma).n()
        print(f"\nWhen beta={nbeta} and gamma={ngamma} we have p={expr.subs(beta=nbeta, gamma=ngamma)}")

The roots provide:

When beta=0.415467947986187 and gamma=0.388623778132756 we have p=0.149614975606356

When beta=0.917865385347146 and gamma=0.388623778132756 we have p=0.146857291142747

When beta=0.398299187168959 and gamma=0.281580084709900 we have p=0.150083407311578

When beta=0.935034146164375 and gamma=0.281580084709900 we have p=0.139693668001026

When beta=0.649497905849455 and gamma=1.14229613715734 we have p=0.118798030775334

When beta=0.683835427483879 and gamma=1.14229613715734 we have p=0.119074926545675

When beta=0.666666666666667 and gamma=0.500000000000000 we have p=0.148148148148148

When beta=0.666666666666667 and gamma=0.500000000000000 we have p=0.148148148148148


## Section 2 - Testing stability

A general script following Theorem 7.1 of Pikhurko–Sliačan–Tyros (2019) is used in subsection 2.1 to verify stability for cases 9-11, and another script tests almost regularity for cases 2 and 15. 

Then, the conditions for flip averse and strictness are tested for cases 4-6 in Subsection 2.2.

### Subsection 2.1 - Perfect stability and almost regularity

In [6]:
from fractions import Fraction
from sage.algebras.combinatorial_theory import _unflatten_matrix
import pickle

# general helper function to have data in sage compatible format
def to_sage(dim, data):
    if dim==0:
        if isinstance(data, Fraction):
            return QQ(data)
        if isinstance(data, float):
            return RR(data)
        return data
    return [to_sage(dim-1, xx) for xx in data]

def check_stability(index, construction_size, construction_edges, ftype):
    violated = False
    file = "certificates/semiind"+str(index)
    print("\n\nchecking stability for " + file)
    G.reset()
    G.printlevel(0)
    if not file.endswith(".pickle"):
        file += ".pickle"
    with open(file, "rb") as f:
        certificate = pickle.load(f)
    target_size = certificate["target size"]
    original_bound = to_sage(0, certificate["result"])
    
    # Checking condition 2.a
    ftype_untyped = ftype.subflag(ftype_points=[])
    if ftype_untyped != G(1):
        G.exclude(ftype_untyped)
        target_2a = get_targ(gen4[index])
        if target_2a==0: 
            bound = 0
        else:
            bound = G.optimize(target_2a, target_size, exact=True, denom=2**20, construction=[])
        G.reset()
    else:
        bound = 0
    if bound < original_bound:
        print(" - condition 2.a is satisfied")
    else:
        print(" - condition 2.a is not satisfied")
        violated = True

    # Checking condition 3
    construction = G.blowup_construction(target_size, construction_size, edges=construction_edges)
    cvals = construction.values()
    svals = to_sage(1, certificate["slack vector"])
    correct_slacks = True
    for ii in range(len(svals)):
        if svals[ii]==0 and cvals[ii]==0:
            correct_slacks = False
    if not correct_slacks:
        print(" - condition 3 is not satisfied. The following flags violate it:")
        for ii,ff in enumerate(G.generate(target_size)):
            if svals[ii]==0 and cvals[ii]==0:
                print(ff)
        violated = True
    else:
        print(" - condition 3 is satisfied")

    # Checking condition i
    index = -1
    for ii,xx in enumerate(certificate["typed flags"]):
        if xx[1] == ftype._pythonize():
            index = ii
            break
    if index==-1:
        print(" - type not found")
        return
    mat = matrix(to_sage(2, _unflatten_matrix(certificate["X matrices"][index])[0]))
    if mat.nullity()==1:
        print(" - condition i is satisfied")
        if not violated:
            print("problem is stable")
        return
    else:
        print(" - condition i is not satisfied, the matrix has nullity", mat.nullity())

    # Checking condition ii
    B_edges = [xx for xx in construction_edges if xx[0]!=xx[1]]
    B = G(construction_size, edges=B_edges)
    G.exclude(B)
    target_ii = get_targ(gen4[index])
    
    if target_ii==0: 
        bound = 0
    else:
        bound = G.optimize(target_ii, target_size, exact=True, denom=2**20, construction=[])
    G.reset()
    if bound < original_bound:
        print(" - condition ii is satisfied")
        if not violated:
            print("problem is stable")
    else:
        print(" - condition ii is not satisfied")

def check_almost_regularity(index):
    file = "certificates/semiind"+str(index)
    print("\n\nchecking almost regularity for " + file)
    G.reset()
    G.printlevel(0)
    if not file.endswith(".pickle"):
        file += ".pickle"
    with open(file, "rb") as f:
        certificate = pickle.load(f)
    target_size = certificate["target size"]
    edge_type = G(2, edges=[[0, 1]], ftype=[0, 1])
    edge_regularity = G(3, edges=[[0, 1], [0, 2]], ftype=[0, 1]) - G(3, edges=[[0, 1], [1, 2]], ftype=[0, 1])
    nonedge_type = G(2, edges=[], ftype=[0, 1])
    nonedge_regularity = G(3, edges=[[0, 2]], ftype=[0, 1]) - G(3, edges=[[1, 2]], ftype=[0, 1])
    
    for ii,xx in enumerate(certificate["typed flags"]):
        fsz = xx[0]
        ftp = xx[1]
        if ftp == edge_type._pythonize():
            X = matrix(to_sage(2, _unflatten_matrix(certificate["X matrices"][ii])[0]))
            reg_vec = (edge_regularity<<(fsz - 3)).values()
            if reg_vec in X.image():
                print(" - almost degree regularity holds between edges")
            else:
                print(" - almost degree regularity fails between edges")
        elif ftp == nonedge_type._pythonize():
            X = matrix(to_sage(2, _unflatten_matrix(certificate["X matrices"][ii])[0]))
            reg_vec = (nonedge_regularity<<(fsz - 3)).values()
            if reg_vec in X.image():
                print(" - almost degree regularity holds between nonedges")
            else:
                print(" - almost degree regularity fails between nonedges")

In [7]:
# Stability checks based on Theorem 2.3
G.reset()
def kn_edges(n):
    return list(itertools.combinations(range(n), 2))
no4b_type = G(5, edges=[xx for xx in kn_edges(5) if xx!=(2, 3)], ftype=[0, 1, 2, 3, 4])
no1_type = G(1, ftype=[0])
ne_type = G(2, edges=[], ftype=[0, 1])

check_stability(9, 5, kn_edges(5), no4b_type)
check_stability(10, 2, [[0, 1]], no1_type)
check_stability(11, 2, [[0, 1]], ne_type)



checking stability for certificates/semiind9
 - condition 2.a is satisfied
 - condition 3 is satisfied
 - condition i is satisfied
problem is stable


checking stability for certificates/semiind10
 - condition 2.a is satisfied
 - condition 3 is satisfied
 - condition i is not satisfied, the matrix has nullity 2
 - condition ii is satisfied
problem is stable


checking stability for certificates/semiind11
 - condition 2.a is satisfied
 - condition 3 is satisfied
 - condition i is satisfied
problem is stable


In [8]:
# Almost regularity checks based on Corollary 2.2
check_almost_regularity(2)
check_almost_regularity(15)



checking almost regularity for certificates/semiind2
 - almost degree regularity holds between nonedges
 - almost degree regularity holds between edges


checking almost regularity for certificates/semiind15
 - almost degree regularity holds between nonedges
 - almost degree regularity holds between edges


In [9]:
no2_type = G(2, ftype=[0, 1])
check_stability(4, 3, kn_edges(3), no2_type)
check_stability(6, 2, [[0, 1]], no2_type)



checking stability for certificates/semiind4
 - condition 2.a is satisfied
 - condition 3 is not satisfied. The following flags violate it:
Flag on 4 points, ftype from () with edges=(01 02 03 12 13 23)
 - condition i is satisfied


checking stability for certificates/semiind6
 - condition 2.a is satisfied
 - condition 3 is not satisfied. The following flags violate it:
Flag on 6 points, ftype from () with edges=(01 02 04 05)
Flag on 6 points, ftype from () with edges=(02 03 05 12 13 15)
 - condition i is not satisfied, the matrix has nullity 4
 - condition ii is satisfied


### Subsection 2.2 - Flip averseness and strictness

In [10]:
import itertools

def check_flip_averse(parts, edges, target):
    n = len(parts) if isinstance(parts, list) else parts
    weights = parts if isinstance(parts, list) else [1/n]*n
    tsize = target.size()
    RB = vector(weights).base_ring()
    R.<eps> = RB[]
    v_new1, v_new2 = n, n + 1
    correct = True
    for pp in itertools.combinations_with_replacement(range(n), 2):
        f1, f2 = pp
        new_weights = list(weights)
        new_weights[f1] -= eps
        new_weights[f2] -= eps
        new_weights.extend([eps, eps])
        original_edges_set = {tuple(sorted(edge)) for edge in edges}
        def get_parent(v):
            if v == v_new1:
                return f1
            if v == v_new2:
                return f2
            return v
        added_edges = set()
        for i in range(n + 2):
            parent_i = get_parent(i)
            parent_new1 = get_parent(v_new1)
            if tuple(sorted([parent_i, parent_new1])) in original_edges_set:
                added_edges.add(tuple(sorted([i, v_new1])))
            parent_new2 = get_parent(v_new2)
            if tuple(sorted([parent_i, parent_new2])) in original_edges_set:
                added_edges.add(tuple(sorted([i, v_new2])))
        edge_to_flip = tuple(sorted([v_new1, v_new2]))
        if edge_to_flip in added_edges:
            added_edges.remove(edge_to_flip)
        else:
            added_edges.add(edge_to_flip)
        new_edges = edges + [list(edge) for edge in added_edges]
        
        expr = G.blowup_construction(tsize, new_weights, edges=new_edges).density(target)
        diff0 = expr.differentiate(eps, 2).subs(eps=0)
        if diff0>=0:
            print("flip averse fails at pair {}, poly is {}".format(pp, expr))
            correct = False
    if correct:
        print("flip averse check successful")
        

def check_strictness(parts, edges, target, optat):
    n = len(parts) if isinstance(parts, list) else parts
    weights = parts if isinstance(parts, list) else [1/n]*n
    tsize = target.size()
    target = target.afae()
    RB = vector(weights + optat).base_ring()
    R = PolynomialRing(RB, n+1, order="lex", names = ["x" + str(ii) for ii in range(n)] + ["z"])
    vars = R.gens()
    Z = vars[-1]
    new_weights = []
    for ii in range(n):
        new_weights += [weights[ii]*vars[ii], weights[ii]*(1-vars[ii])]
    new_edges = []
    for e0, e1 in edges:
        d0, d1 = e0*2, e1*2
        new_edges += [[d0, d1], [d0, d1+1], [d0+1, d1+1]]
        if e0!=e1:
            new_edges.append([d0+1, d1])
    marked = [[2*ii] for ii in range(n)]
    CG = combine("ColoredGraph", G, Color0)
    param_cons = CG.blowup_construction(tsize-1, new_weights, edges=new_edges, C0=marked)
    expr = R(0)

    def attach(cgflag):
        nn = cgflag.size() + 1
        neds = list(cgflag.blocks("edges"))
        neds += [[xx[0], nn-1] for xx in cgflag.blocks("C0")]
        return G(nn, edges=neds)

    for vv, cfl in param_cons:
        expr += target[attach(cfl)] * vv

    optimum = expr.subs({vars[ii]:optat[ii] for ii in range(n)})
    ders = [expr.derivative(xx) for xx in vars[:-1]]
    zexpr = Z - expr
    sat_expr = "".join(sorted([str(xx) if xx in [0, 1] else "*" for xx in optat]))
    correct = True
    for ppos in itertools.product(["*", "0", "1"], repeat=n):
        subs = {vars[ii]: QQ(ppos[ii]) for ii in range(n) if ppos[ii]!="*"}
        ids = [ders[ii].subs(subs) for ii in range(n) if ppos[ii]=="*"] + [zexpr.subs(subs)]
        I = R.ideal(ids)
        Gb = I.groebner_basis()
        if Gb[-1].subs({Z: optimum})==0:
            osat_expr = "".join(sorted(ppos))
            if osat_expr != sat_expr:
                print("strictness fails at attachments {}".format(str(ppos)))
                correct = False
    if correct:
        print("strictness check successful")

In [11]:
check_strictness(3, kn_edges(3), get_targ(gen4[4]), [1, 0, 0])
check_flip_averse(3, kn_edges(3), get_targ(gen4[4]))

strictness check successful
flip averse check successful


In [12]:
check_strictness(2, [[0, 1]], get_targ(gen4[6]), [1, 0])
check_flip_averse(2, [[0, 1]], get_targ(gen4[6]))

strictness fails at attachments ('0', '0')
flip averse check successful


In [13]:
var("x")
RF = RealField(prec=100)
alpha_real = RF(solve(x^2 - 13/7*x + 4/7==0, x)[0].rhs())
R.<alpha> = NumberField(x^2 - 13/7*x + 4/7, embedding=alpha_real)
targ = get_targ(gen4[5])
check_strictness([alpha/4, alpha/4, alpha/4, alpha/4, 1-alpha], list(itertools.combinations(range(5), 2)), targ, [0, 1, 1, 1, 1])
check_flip_averse([alpha/4, alpha/4, alpha/4, alpha/4, 1-alpha], list(itertools.combinations(range(5), 2)), targ)

strictness check successful
flip averse check successful


## Section 3 - Verify certificates

If one only wants to verify that any of the exact certificates are indeed correct, it is enough
to run this cell. For each step above, it loads the generated certificates and verifies that the matrices are indeed positive semidefinite
and that the bound they prove is exactly as claimed. Note however that the majority of the time is spent at calculating the problem data, once that is complete all the cells run quickly and it is not much slower to re-calculate the certificates instead of verifying them.

In [6]:
G = GraphTheory
G.printlevel(1)
BG = Theory("BlueGraph", "blue_edges")
CG = combine("RedBlueGraph", G, BG, symmetries=FullSymmetry)
CG.exclude(CG(2, edges=[[0, 1]], blue_edges=[[0, 1]]))
def check_blocks(flg, do, dont):
    eds = flg.blocks("edges")
    for ee in do:
        if ee not in eds:
            return False
    for ee in dont:
        if ee in eds:
            return False
    return True
def get_targ(ff):
    vn = ff.size()
    eddo = ff.blocks("edges")
    eddont = ff.blocks("blue_edges")
    pats = G.p(vn, edges=eddo, edges_m=eddont)
    targ = 0
    for xx in pats.compatible_flags():
        coef = 0
        for eperm in itertools.permutations(range(vn)):
            xxpermed = xx.subflag(eperm)
            if check_blocks(xxpermed, eddo, eddont):
                coef += 1
        targ += coef*xx
    return targ*QQ(1/(factorial(vn)))
def gen_nont(n):
    ret = []
    for ff in CG.generate(n):
        eds = ff.blocks("edges")
        oeds = ff.blocks("blue_edges")
        if len(eds)!=0 and len(oeds)!=0:
            if len(eds) + len(oeds) != binomial(n, 2):
                if len(set(itertools.chain(*(list(eds) + list(oeds)))))==n:
                    ret.append(ff)
    return ret
def flip_colors(fl):
    return CG(fl.size(), edges=fl.blocks("blue_edges"), blue_edges=fl.blocks("edges"))
gen4 = [(xx if ii not in [4, 8, 9, 10, 14, 17] else flip_colors(xx)) for ii,xx in enumerate(gen_nont(4))]
for ii,xx in enumerate(gen4):
    if ii in [3, 5]:
        continue
    print(f"\nTesting case {ii} with {gen4[ii]}")
    G.verify("certificates/semiind"+str(ii))


Testing case 0 with Flag on 4 points, ftype from () with edges=(02), blue_edges=(13)
Checking X matrices


5it [00:00, 6837.80it/s]


Solution matrices are all positive semidefinite, linear coefficients are all non-negative
Calculating multiplication tables


5it [00:00, 3084.05it/s]


Done calculating linear constraints
Calculating the bound provided by the certificate


...

Solution matrices are all positive semidefinite, linear coefficients are all non-negative
Calculating multiplication tables


5it [00:00, 4206.92it/s]


Done calculating linear constraints
Calculating the bound provided by the certificate


5it [00:00, 416.39it/s]

The solution is valid, it proves the bound 1/8


## Section 4 - Inspect certificates interactively

While largest semi-definite block has dimension 72, most of the obtained results use smaller calculations. This last part allows the interactive inspection of the certificate. An easy to read text file is also exported to the certificates folder, for each exact result.

First, an easy to read data is exported. They are simple text files, showing the linear combination of typed configurations, whose (scaled) square provides the final proof in each case.

These terms can be inspected afterwards, in an interactive way as well, by running the final cell.

In [14]:
import pickle
from sage.algebras.combinatorial_theory import _unflatten_matrix
G = GraphTheory
G.printlevel(0)

def hprint(hflag):
    vnum = hflag.size()
    eds = hflag.blocks("edges")
    seds = "(" + " ".join([str(uu)+str(vv) for uu,vv in eds]) + ")"
    type = hflag.ftype_points()
    return f"({vnum} vertex graph with {seds} edges and {type} type)"
def vecprint(vec, tdata, hide=True):
    flags = G.generate(tdata[0], tdata[1])
    return "\n".join([f"{ww} \t {hprint(flags[ii])}" for ii,ww in enumerate(vec) if ww!=0 or (not hide)])

interactive_data = {}

for nind in range(18):
    name = "semiind" + str(nind)
    fp = False
    try:
        cert = pickle.load(open(f"certificates/{name}.pickle", "rb"))
        try:
            bound = QQ(cert["result"])
        except:
            bound = cert["result"]
    except:
        cert = pickle.load(open(f"certificates/fp_{name}.pickle", "rb"))
        fp = True
        bound = RR(cert["result"])
    tsize = cert["target size"]
    base_strs = G.generate(tsize)
    base_hprint = "\n".join(map(hprint, base_strs))
    slacks_hprint = vecprint(vector(cert["slack vector"]), [tsize, None], False)
    slacks = cert["slack vector"]
    int_data = {
        "0 - the final bound": bound,
        "1 - the vertices of the graphs used": tsize, 
        "2 - the list of maximum sized graphs used": base_hprint, 
        "3 - the list of slack values": slacks_hprint
    }
    squares = {}
    with open(f"certificates/{name}.txt", "w") as file:
        file.write(f"Text certificate for {name} proving the bound {bound}\n{"="*50}\n\n")
        file.write(f"Base structures\n{base_hprint}\n{"="*50}\n\n")
        file.write(f"Slack values\n{slacks_hprint}\n{"="*50}\n\n")
        file.write(f"Semi-definite blocks\n")
        for ii,tdata in enumerate(cert["typed flags"].keys()):
            tdata = [tdata[0], G(tdata[1][0], ftype=tdata[1][1], **dict(tdata[1][2]))]
            mat = _unflatten_matrix(cert["X matrices"][ii])[0]
            P, L, D = mat.block_ldlt()
            type_hprint = hprint(tdata[1])
            type_interactive = {}
            file.write(f"\n{"-"*50}\n\n\nType {type_hprint} contributes the following")
            all_squares = ""
            for jj,col in enumerate((P*L).columns()):
                text = f"\n{D[jj,jj]} scaled square of \n{vecprint(col, tdata)}\n\n"
                all_squares += text
                type_interactive[f"{jj} - the square with this index and it's scale"] = text
                if D[jj,jj]!=0:
                    file.write(text)
            type_interactive[f"{P.nrows()} - all the squares and their scale"] = all_squares
            squares[f"{ii} - squares coming from type {type_hprint}"] = type_interactive
    int_data["4 - the squares used in the proof"] = squares
    interactive_data[f"{nind} - certificate for {name}"] = int_data

In [15]:
def inspect(dd):
    keys = list(dd.keys())
    for kk in keys:
        print(kk)
    print("\n")
    val = input("Inspect: ")
    res = dd[keys[int(val)]]
    print("\n")
    if isinstance(res, dict):
        inspect(res)
    else:
        print(res)

inspect(interactive_data)

0 - certificate for semiind0
1 - certificate for semiind1
2 - certificate for semiind2
3 - certificate for semiind3
4 - certificate for semiind4
5 - certificate for semiind5
6 - certificate for semiind6
7 - certificate for semiind7
8 - certificate for semiind8
9 - certificate for semiind9
10 - certificate for semiind10
11 - certificate for semiind11
12 - certificate for semiind12
13 - certificate for semiind13
14 - certificate for semiind14
15 - certificate for semiind15
16 - certificate for semiind16
17 - certificate for semiind17




Inspect:  4




0 - the final bound
1 - the vertices of the graphs used
2 - the list of maximum sized graphs used
3 - the list of slack values
4 - the squares used in the proof




Inspect:  4




0 - squares coming from type (2 vertex graph with () edges and (0, 1) type)
1 - squares coming from type (2 vertex graph with (01) edges and (0, 1) type)




Inspect:  0




0 - the square with this index and it's scale
1 - the square with this index and it's scale
2 - the square with this index and it's scale
3 - the square with this index and it's scale
4 - all the squares and their scale




Inspect:  4





4/27 scaled square of 
1 	 (3 vertex graph with () edges and (0, 1) type)
-173799/1048576 	 (3 vertex graph with (01) edges and (0, 2) type)
-173799/1048576 	 (3 vertex graph with (01) edges and (2, 0) type)
-1/2 	 (3 vertex graph with (01 02) edges and (1, 2) type)


80519713181/274877906944 scaled square of 
1 	 (3 vertex graph with (01) edges and (0, 2) type)
-43858177635/80519713181 	 (3 vertex graph with (01) edges and (2, 0) type)


271790293770271/1319234980757504 scaled square of 
1 	 (3 vertex graph with (01) edges and (2, 0) type)


0 scaled square of 
1 	 (3 vertex graph with (01 02) edges and (1, 2) type)


